In [1]:
import sys
import os

import pandas as pd

app_base_dir = '/home/spbproc/euso-spb-patt-reco-v1'
if app_base_dir not in sys.path:
    sys.path.append(app_base_dir)

import event_processing_v3
import event_processing_v4
import postgresql_v3_event_storage
import dataset_query_functions_v3

import sqlite3

In [2]:
classifiaction_tables_dir = 'szakacs_external_cnn_classification_results_20190522'
data_snippets_dir = 'szakacs_external_cnn_classification_results_20190522'

os.makedirs(data_snippets_dir, exist_ok=True)

In [3]:
event_v3_storage_provider_flight = dataset_query_functions_v3.build_event_v3_storage_provider(
    event_storage_provider_config_file=os.path.join(app_base_dir,'config_w_flatmap.ini'), 
    table_names_version='ver4',
    event_storage_class=postgresql_v3_event_storage.PostgreSqlEventV3StorageProvider,
    event_processing_class=event_processing_v4.EventProcessingV4,
    readonly=False
)

query_functions_flight = dataset_query_functions_v3.Ver3DatasetQueryFunctions(event_v3_storage_provider_flight)

In [4]:
# %load_ext autoreload
# %autoreload 2

In [5]:
sqlite_db_pathname = os.path.join(classifiaction_tables_dir, 'flight_evals.sqlite')

In [6]:
conn = sqlite3.connect(sqlite_db_pathname)

In [7]:
classification_columns = (('event_id', int), ('class', int), ('class_probability', float))

## tnet_simu

In [8]:
classification_slug_tnet_simu = 'cnn_tnet_simu_20190521'

In [9]:
tnet_simu_df = pd.read_sql('SELECT * FROM tnet_simu', conn)

In [10]:
tnet_simu_df.head()

,item_idx,output,target,shower_prob,noise_prob,source_file_acquisition_full,start_gtu,end_gtu,packet_id,event_id
0,0,noise,noise,0.0,1.0,/home/spbproc/SPBDATA_flight/allpackets-SPBEUS...,35,55,0,61
1,1,noise,noise,0.0,1.0,/home/spbproc/SPBDATA_flight/allpackets-SPBEUS...,34,54,1,67
2,2,noise,noise,0.0,1.0,/home/spbproc/SPBDATA_flight/allpackets-SPBEUS...,35,55,0,73
3,3,shower,noise,1.0,0.0,/home/spbproc/SPBDATA_flight/allpackets-SPBEUS...,30,50,1,75
4,4,noise,noise,0.0,1.0,/home/spbproc/SPBDATA_flight/allpackets-SPBEUS...,34,54,0,77


In [11]:
tnet_simu_df['class'] = tnet_simu_df['output'].str.match('shower').astype('int32')

In [12]:
tnet_simu_df[['event_id', 'class', 'shower_prob']].head()

,event_id,class,shower_prob
0,61,0,0.0
1,67,0,0.0
2,73,0,0.0
3,75,1,1.0
4,77,0,0.0


In [13]:
event_v3_storage_provider_flight.create_classification_table(
    classification_slug_tnet_simu, classification_columns)

In [14]:
event_v3_storage_provider_flight.save_classification_data(
    classification_slug_tnet_simu, tnet_simu_df[['event_id', 'class', 'shower_prob']].values, 
    classification_columns, num_inserts_at_once=1000)

## gnet3b_simu

In [15]:
classification_slug_gnet3b_simu = 'cnn_gnet3b_simu_20190521'

In [16]:
gnet3b_simu_df = pd.read_sql('SELECT * FROM gnet3b_simu', conn)

In [17]:
gnet3b_simu_df.head()

,item_idx,output,target,shower_prob,noise_prob,source_file_acquisition_full,start_gtu,end_gtu,packet_id,event_id
0,0,noise,noise,0.0,1.0,/home/spbproc/SPBDATA_flight/allpackets-SPBEUS...,35,55,0,61
1,1,noise,noise,0.0,1.0,/home/spbproc/SPBDATA_flight/allpackets-SPBEUS...,34,54,1,67
2,2,noise,noise,0.0,1.0,/home/spbproc/SPBDATA_flight/allpackets-SPBEUS...,35,55,0,73
3,3,noise,noise,0.0,1.0,/home/spbproc/SPBDATA_flight/allpackets-SPBEUS...,30,50,1,75
4,4,noise,noise,0.0,1.0,/home/spbproc/SPBDATA_flight/allpackets-SPBEUS...,34,54,0,77


In [18]:
gnet3b_simu_df['class'] = gnet3b_simu_df['output'].str.match('shower').astype('int32')

In [19]:
gnet3b_simu_df[['event_id', 'class', 'shower_prob']].head()

,event_id,class,shower_prob
0,61,0,0.0
1,67,0,0.0
2,73,0,0.0
3,75,0,0.0
4,77,0,0.0


In [20]:
event_v3_storage_provider_flight.create_classification_table(
    classification_slug_gnet3b_simu, classification_columns)

In [21]:
event_v3_storage_provider_flight.save_classification_data(
    classification_slug_gnet3b_simu, gnet3b_simu_df[['event_id', 'class', 'shower_prob']].values, 
    classification_columns, num_inserts_at_once=1000)